# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900299  89.199585  130.451974  89.487615  127.713291   
2022-12-27  128.097473  87.332367  129.456968  88.891696  126.806945   
2022-12-28  124.166779  85.872360  129.082614  87.918357  123.999307   
2022-12-29  127.683731  88.345428  128.540796  88.757609  125.831674   
2022-12-30  127.998978  88.126923  128.018685  88.226241  125.536140   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  87.024475  128.974267  87.024475  63814900  17815000  
2022-12-27  86.940055  129.427415  88.702985  69007800  15470900  
2022-12-28  85.782976  127.742826  86.905293  85438400  17879600  
2022-12-29  86.398751  126.087805  86.438480  75703700  18280700  
2022-12-30  86.438473  126.501579  86.771195  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

Strategy started!
Bought 1239 AAPL and 945 GOOG
Strategy finished!


In [5]:
result.head()

,ITERATION,BALANCE,POSITIONS_TOTAL,EQUITY
0,2018-01-02,37.17601,99962.823990,100000.000000
1,2018-01-03,37.17601,100774.439255,100811.615265
2,2018-01-04,37.17601,101190.535480,101227.711491
3,2018-01-05,37.17601,102505.045246,102542.221256
4,2018-01-08,37.17601,102537.458565,102574.634575


## <b> Backtest Statistics </b>

In [6]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 37.18
Final equity: 241907.85
---------------------------


In [7]:
bt.statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'avg_abs_return': np.float64(141907.85119628906),
 'avg_abs_return_per_win': np.float64(141907.85119628906),
 'avg_abs_return_per_lost': nan,
 'median_abs_return': np.float64(141907.85119628906),
 'median_abs_return_per_win': np.float64(141907.85119628906),
 'median_abs_return_per_lost': nan,
 'std_return': np.float64(nan),
 'mad_return': np.float64(0.0),
 'downside_deviation': np.float64(0.0),
 'upside_deviation': np.float64(141907.85119628906),
 'skewness': np.float64(nan),
 'kurtosis': np.float64(nan),
 'mean_median_gap': np.float64(0.0)}

In [10]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,ada989c5-724f-4aee-b858-9e024e2c04b1,OPEN,Buy and Hold Trade,141907.851196,2018-01-02,None,None,241870.675186,None,None,None


In [11]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL
0,b417c333-1109-43fa-9b95-fe73dc466f4f,AAPL,BUY,FILLED,ada989c5-724f-4aee-b858-9e024e2c04b1,2018-01-02,1239,40.341885,0.0,49983.595036,49983.595036,2018-01-02,1239,40.341885,0.0,-49983.595036,-49983.595036
1,65e00124-5c01-4cf4-b2d8-79a9fa0dd23d,GOOG,BUY,FILLED,ada989c5-724f-4aee-b858-9e024e2c04b1,2018-01-02,945,52.888073,0.0,49979.228954,49979.228954,2018-01-02,945,52.888073,0.0,-49979.228954,-49979.228954


In [12]:
bt.strategy.positions

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.73332214355)},
 'GOOG': {'quantity': 945, 'value': np.float64(83279.94186401367)}}